In [1]:
import xarray as xr
import cartopy
from matplotlib import pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np
from matplotlib.collections import PatchCollection
from dask.distributed import Client, wait
from matplotlib.patches import Polygon
#import cmocean
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [2]:
# parameters
connectivity_file_path = "/gxfs_work/geomar/smomw400/git_projects/2022_north-sea_oysters/output_data/035_connection_count_05m/"
connectivity_file_pattern = "035_connection_count_05m_days-travelled-??_bio_phys_agg.nc"

netcdf_file_path = Path("/gxfs_work/geomar/smomw400/git_projects/2022_north-sea_oysters/output_data/040_connectivity_analysis_05m")

In [3]:
connectivity_file_list = sorted(Path(connectivity_file_path).glob(connectivity_file_pattern))
len(connectivity_file_list)

29

In [6]:
####DASK
#client = Client(n_workers=8, threads_per_worker=4, memory_limit=32e9, ip="0.0.0.0")
client = Client(scheduler_file="/gxfs_work/geomar/smomw400/git_projects/2022_north-sea_oysters/notebooks/manuscript/scheduler.json")
#client.re.start()
client

Connection method: Scheduler file,Scheduler file: /gxfs_work/geomar/smomw400/git_projects/2022_north-sea_oysters/notebooks/manuscript/scheduler.json
Dashboard: http://10.0.4.175:8787/status,
Comm: tcp://10.0.4.175:8786,Workers: 0
Dashboard: http://10.0.4.175:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [7]:
def load_and_clean_connectivity_files(connectivity_file_list, chunks=None):
    if chunks is None:
        chunks={"hex0": 1000, "month": 1, "year": 1}

    ds = xr.concat(
        [
            xr.open_dataset(_df, chunks=chunks).assign_coords(days_travelled=_dt)
            for _dt, _df in enumerate(connectivity_file_list)
        ],
        dim="days_travelled"
    )
    
    for coord in ds.coords:
        if "days_travelled" in ds[coord].dims and len(ds[coord].dims) > 1:
            ds[coord] = ds[coord].mean("days_travelled").compute()

    for var in ds.data_vars:
        if (var.endswith("_hex0") or var.endswith("_hex1")) and ("days_travelled" in ds[var].dims):
            ds[var] = ds[var].mean("days_travelled").compute()

    return ds

In [8]:
ds = load_and_clean_connectivity_files(connectivity_file_list)

In [9]:
ds

<xarray.Dataset> Size: 326GB
Dimensions:              (hex0: 8364, hex1: 8397, days_travelled: 29, month: 5,
                          year: 4, corner: 7)
Coordinates: (12/14)
  * hex0                 (hex0) <U14 468kB '(-1, -19, 20)' ... '(9, 9, -18)'
  * hex1                 (hex1) <U14 470kB '(-1, -19, 20)' ... '(9, 9, -18)'
  * month                (month) float64 40B 0.0 1.0 2.0 3.0 4.0
  * year                 (year) float64 32B 0.0 1.0 2.0 3.0
    lon_hex0_corners     (corner, hex0) float64 468kB -3.072 -3.076 ... -0.6406
    lat_hex0_corners     (corner, hex0) float64 468kB 51.46 54.11 ... 56.58
    ...                   ...
    lon_hex0             (hex0) float64 67kB -3.216 -3.229 ... -0.8121 -0.8032
    lon_hex1             (hex1) float64 67kB -3.216 -3.229 ... -0.8121 -0.8032
    lat_hex0             (hex0) float64 67kB 51.46 54.11 51.31 ... 56.43 56.58
    lat_hex1             (hex1) float64 67kB 51.46 54.11 51.31 ... 56.43 56.58
    hex_label            (hex0) <U14 468kB '(-1, -19, 20)' ... '(9, 9, -18)'
  * days_travelled       (days_travelled) int64 232B 0 1 2 3 4 ... 25 26 27 28
Dimensions without coordinates: corner
Data variables: (12/25)
    aqc_count_hex0       (hex0) float64 67kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    rst_count_hex0       (hex0) float64 67kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    pop_count_hex0       (hex0) float64 67kB 1.0 1.0 6.0 0.0 ... 0.0 0.0 0.0 0.0
    dss_count_hex0       (hex0) float64 67kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    hly_count_hex0       (hex0) float64 67kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    his_count_hex0       (hex0) float64 67kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...                   ...
    depth_mean_hex0      (hex0) float32 33kB 6.895 7.289 16.08 ... 60.0 60.0
    depth_mean_hex1      (hex1) float32 34kB 6.895 7.289 16.08 ... 60.0 60.0
    depth_median_hex0    (hex0) float64 67kB 5.0 5.0 15.0 ... 60.0 60.0 60.0
    depth_median_hex1    (hex1) float64 67kB 5.0 5.0 15.0 ... 60.0 60.0 60.0
    depth_std_hex0       (hex0) float32 33kB 3.161 3.981 4.406 ... 2.678 0.0 0.0
    depth_std_hex1       (hex1) float32 34kB 3.161 3.981 4.406 ... 2.678 0.0 0.0
Attributes:
    hex_proj__lon_origin:       -3.0
    hex_proj__lat_origin:       54.5
    hex_proj__hex_size_meters:  10000

### Select days 00-07 - Bonamia

In [ ]:
ds_conn = ds.sel(days_travelled=slice(None, 7)).sum("days_travelled")

<xarray.Dataset> Size: 11GB
Dimensions:              (hex0: 8364, hex1: 8397, month: 5, year: 4, corner: 7)
Coordinates: (12/13)
  * hex0                 (hex0) <U14 468kB '(-1, -19, 20)' ... '(9, 9, -18)'
  * hex1                 (hex1) <U14 470kB '(-1, -19, 20)' ... '(9, 9, -18)'
  * month                (month) float64 40B 0.0 1.0 2.0 3.0 4.0
  * year                 (year) float64 32B 0.0 1.0 2.0 3.0
    lon_hex0_corners     (corner, hex0) float64 468kB -3.072 -3.076 ... -0.6406
    lat_hex0_corners     (corner, hex0) float64 468kB 51.46 54.11 ... 56.58
    ...                   ...
    lat_hex1_corners     (corner, hex1) float64 470kB 51.46 54.11 ... 56.58
    lon_hex0             (hex0) float64 67kB -3.216 -3.229 ... -0.8121 -0.8032
    lon_hex1             (hex1) float64 67kB -3.216 -3.229 ... -0.8121 -0.8032
    lat_hex0             (hex0) float64 67kB 51.46 54.11 51.31 ... 56.43 56.58
    lat_hex1             (hex1) float64 67kB 51.46 54.11 51.31 ... 56.43 56.58
    hex_label            (hex0) <U14 468kB '(-1, -19, 20)' ... '(9, 9, -18)'
Dimensions without coordinates: corner
Data variables: (12/25)
    aqc_count_hex0       (hex0) float64 67kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    rst_count_hex0       (hex0) float64 67kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    pop_count_hex0       (hex0) float64 67kB 1.0 1.0 6.0 0.0 ... 0.0 0.0 0.0 0.0
    dss_count_hex0       (hex0) float64 67kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    hly_count_hex0       (hex0) float64 67kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    his_count_hex0       (hex0) float64 67kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...                   ...
    depth_mean_hex0      (hex0) float32 33kB 6.895 7.289 16.08 ... 60.0 60.0
    depth_mean_hex1      (hex1) float32 34kB 6.895 7.289 16.08 ... 60.0 60.0
    depth_median_hex0    (hex0) float64 67kB 5.0 5.0 15.0 ... 60.0 60.0 60.0
    depth_median_hex1    (hex1) float64 67kB 5.0 5.0 15.0 ... 60.0 60.0 60.0
    depth_std_hex0       (hex0) float32 33kB 3.161 3.981 4.406 ... 2.678 0.0 0.0
    depth_std_hex1       (hex1) float32 34kB 3.161 3.981 4.406 ... 2.678 0.0 0.0

In [ ]:
ds_conn.to_netcdf(str(netcdf_file_path / "040_connectivity_analysis_05m_00-07days.nc"))

### Select days 07-14 - fast development

In [12]:
ds_conn14 = ds.sel(days_travelled=slice(7, 14)).sum("days_travelled")

In [ ]:
ds_conn14.to_netcdf(str(netcdf_file_path / "040_connectivity_analysis_05m_07-14days.nc"))

### Select days 07-28 - slow development

In [10]:
ds_conn28 = ds.sel(days_travelled=slice(7, None)).sum("days_travelled")

In [11]:
ds_conn28.to_netcdf(str(netcdf_file_path/"040_connectivity_analysis_05m_07-28days.nc"))

/opt/conda/envs/parcels/lib/python3.12/site-packages/distributed/client.py:3358: UserWarning: Sending large graph of size 15.95 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [17]:

file_path = "/gxfs_work/geomar/smomw400/git_projects/2022_north-sea_oysters/output_data/040_connectivity_analysis_05m/040_connectivity_analysis_05m_07-28days.nc"
ds = xr.open_dataset(file_path)
ds

<xarray.Dataset> Size: 11GB
Dimensions:              (hex0: 8364, hex1: 8397, month: 5, year: 4, corner: 7)
Coordinates: (12/13)
  * hex0                 (hex0) <U14 468kB '(-1, -19, 20)' ... '(9, 9, -18)'
  * hex1                 (hex1) <U14 470kB '(-1, -19, 20)' ... '(9, 9, -18)'
  * month                (month) float64 40B 0.0 1.0 2.0 3.0 4.0
  * year                 (year) float64 32B 0.0 1.0 2.0 3.0
    lon_hex0_corners     (corner, hex0) float64 468kB ...
    lat_hex0_corners     (corner, hex0) float64 468kB ...
    ...                   ...
    lat_hex1_corners     (corner, hex1) float64 470kB ...
    lon_hex0             (hex0) float64 67kB ...
    lon_hex1             (hex1) float64 67kB ...
    lat_hex0             (hex0) float64 67kB ...
    lat_hex1             (hex1) float64 67kB ...
    hex_label            (hex0) <U14 468kB ...
Dimensions without coordinates: corner
Data variables: (12/25)
    aqc_count_hex0       (hex0) float64 67kB ...
    rst_count_hex0       (hex0) float64 67kB ...
    pop_count_hex0       (hex0) float64 67kB ...
    dss_count_hex0       (hex0) float64 67kB ...
    hly_count_hex0       (hex0) float64 67kB ...
    his_count_hex0       (hex0) float64 67kB ...
    ...                   ...
    depth_mean_hex0      (hex0) float32 33kB ...
    depth_mean_hex1      (hex1) float32 34kB ...
    depth_median_hex0    (hex0) float64 67kB ...
    depth_median_hex1    (hex1) float64 67kB ...
    depth_std_hex0       (hex0) float32 33kB ...
    depth_std_hex1       (hex1) float32 34kB ...